# Pregunta 3
Eliminar filas duplicadas basadas en columnas específicas

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [23]:
spark = SparkSession.builder\
    .appName("Pregunta3")\
    .getOrCreate()
# df_employees_duplicates
df_ed = spark.read.csv('./datasets/employees_duplicates.csv', header=True, inferSchema=True)

In [24]:
columnas = ["Name","Age","Gender","Department","Salary","Joining","Date",
       "Performance","Score","Location","Session"]

In [25]:
# Simple
df_clean = df_ed.dropDuplicates(columnas)
df_clean2 = df_ed.dropDuplicates(['Name','Department','Joining'])
df_clean.show(10)

+--------------------+---+------+----------+------+----------+----+-----------+--------+-----------+-------+
|                Name|Age|Gender|Department|Salary|   Joining|Date|Performance|   Score|   Location|Session|
+--------------------+---+------+----------+------+----------+----+-----------+--------+-----------+-------+
|           Eric York| 34|  Male|        HR|  3397|2022-06-25| 4.0|          2|  Active|Los Angeles|Evening|
|       Mario Jackson| 43|  Male|        IT|  8563|2017-12-28| 4.0|         11|  Active|Los Angeles|  Night|
|Mrs. Kimberly Woo...| 48|Female|     Sales|  7287|2019-03-06| 1.0|          9|  Active|Los Angeles|  Night|
|       Briana Martin| 59|  Male|     Sales|  4496|2023-05-22| 1.0|         20|Inactive|   New York|  Night|
|    Valerie Guerrero| 23|  Male|        HR|  6253|2023-09-21| 4.0|         14|  Active|Los Angeles|  Night|
|    Monica Henderson| 39|  Male|        IT|  7250|2016-07-11| 4.0|          1|  Active|    Chicago|Morning|
|        James Melt

In [27]:
# Windows Functions
from pyspark.sql.window import Window 
window_spec = Window.partitionBy(['Name', 'Department']).orderBy(F.col("Joining").desc()) 
df_clean_wf = df_ed.withColumn("row_num", F.row_number().over(window_spec)) \
              .filter(F.col("row_num") == 1) \
              .drop("row_num") 



In [28]:
print("Cantidad de filas originales:", df_ed.count())
print("Cantidad de filas después de eliminar duplicados (método simple):", df_clean.count())
print("Cantidad de filas después de eliminar duplicados (método con funciones de ventana):", df_clean_wf.count())

Cantidad de filas originales: 1274
Cantidad de filas después de eliminar duplicados (método simple): 1000
Cantidad de filas después de eliminar duplicados (método con funciones de ventana): 1000
